In [1]:
import numpy as np
import xarray as xr
import plotly.graph_objects as go


In [2]:
def mapping_map_to_sphere(lon, lat, radius=1):
    # this function maps the points of coords (lon, lat) to points onto the sphere of radius radius
    lon=np.array(lon, dtype=np.float64)
    lat=np.array(lat, dtype=np.float64)
    lon=np.radians(lon)
    lat=np.radians(lat)
    xs=radius*np.cos(lon)*np.cos(lat)
    ys=radius*np.sin(lon)*np.cos(lat)
    zs=radius*np.sin(lat)
    return xs, ys, zs

In [3]:
# Import topography data
# Select the area you want
resolution = 0.8
lon_area = [-180., 180.]
lat_area = [-90., 90.]
# Get mesh-shape topography data

ds = xr.open_dataarray('/Users/simon/Data/GMTdata/hawaii2017/earth_relief_10m.grd')

lon_topo, lat_topo = np.meshgrid(ds.lon.data, ds.lat.data)
topo = ds.data*0.00001
#lon_topo, lat_topo, topo = Etopo(lon_area, lat_area, resolution)

In [4]:
xs, ys, zs = mapping_map_to_sphere(lon_topo.flatten(), lat_topo.flatten())


In [5]:
Ctopo = [[0, 'rgb(0, 0, 70)'],[0.2, 'rgb(0,90,150)'], 
          [0.4, 'rgb(150,180,230)'], [0.5, 'rgb(210,230,250)'],
          [0.50001, 'rgb(0,120,0)'], [0.57, 'rgb(220,180,130)'], 
          [0.65, 'rgb(120,100,0)'], [0.75, 'rgb(80,70,0)'], 
          [0.9, 'rgb(200,200,200)'], [1.0, 'rgb(255,255,255)']]
cmin = -8000
cmax = 8000

In [6]:
topo_sphere=dict(type='surface',
  x=xs,
  y=ys,
  z=zs,
  colorscale=Ctopo,
  surfacecolor=topo,
  cmin=cmin,
  cmax=cmax)

In [7]:
noaxis=dict(showbackground=False,
  showgrid=False,
  showline=False,
  showticklabels=False,
  ticks='',
  title='',
  zeroline=False)

In [8]:
titlecolor = 'white'
bgcolor = 'black'

layout = go.Layout(
  autosize=False, width=1200, height=800,
  title = '3D spherical topography map',
  titlefont = dict(family='Courier New', color=titlecolor),
  showlegend = False,
  scene = dict(
    xaxis = noaxis,
    yaxis = noaxis,
    zaxis = noaxis,
    aspectmode='manual',
    aspectratio=go.layout.scene.Aspectratio(
      x=1, y=1, z=1)),
  paper_bgcolor = bgcolor,
  plot_bgcolor = bgcolor)

In [ ]:
def spheres(): 
    
    # Set up 100 points. First, do angles
    theta = np.linspace(0,2*np.pi,100)
    phi = np.linspace(0,np.pi,100)
    
    # Set up coordinates for points on the sphere
    x0 = xs.reshape(topo.shape) # dist + size * np.outer(np.cos(theta),np.sin(phi))
    y0 = ys.reshape(topo.shape) # size * np.outer(np.sin(theta),np.sin(phi))
    z0 = zs.reshape(topo.shape) # size * np.outer(np.ones(100),np.cos(phi))
    c0 = topo
    
    #print(x0.shape, c0.shape)
    
    # Set up trace
    trace = go.Surface(x=x0, y=y0, z=z0, 
                       surfacecolor=topo, 
                       colorscale='viridis')
    trace.update(showscale=False)

    return trace

trace0=spheres() # Sun


layout=go.Layout(title = 'Earth', 
                 showlegend=False, 
                 autosize=False,
                 height=1000,
                 width=1200,
                 margin=dict(l=0, r=0, t=0, b=0),
                  #paper_bgcolor = 'black',
                  scene = dict(xaxis=dict(range=[-1,1], 
                                          #backgroundcolor='black',
                                          color='black',
                                          gridcolor='black'),
                               yaxis=dict(range=[-1,1],
                                          #backgroundcolor='black',
                                          color='black',
                                          gridcolor='black'
                                          ),
                               zaxis=dict(range=[-1,1],
                                          #backgroundcolor='black',
                                          color='white', 
                                          gridcolor='black'
                                         ),
                               ))

fig = go.Figure(data = [trace0], layout = layout)

fig.show()
